In [245]:
import osmnx as ox
from geopy.geocoders import Nominatim
import os
import math
import networkx as nx
import folium

In [246]:
distances = dict()
path = dict()

visited = set()

edges = dict()

In [247]:
def Dijkstra(a):
    distances[a] = 0
    
    while True:
        
        curr_key = ""
        curr_min = math.inf
            
        for i in distances.keys():    
                
            if distances[i] >= curr_min or i in visited:
                continue
                
            curr_key = i
            curr_min = distances[i]
        
        if curr_key == "":
            break       
            
        for neighbour, weight in edges[curr_key]:
            distances[str(neighbour)] = min(distances[str(neighbour)], distances[str(curr_key)] + weight)
            
            if distances[str(neighbour)] == distances[str(curr_key)] + weight:
                    path[str(neighbour)] = path[str(curr_key)] + str(neighbour) + "->"
        
        visited.add(curr_key)
    
    return 

In [248]:
ox.config(log_console=True, use_cache=True)
locator = Nominatim(user_agent="myapp")

def_s_pos = "Московская, 158, Саратов, Саратовская область"
def_e_pos = "Волжская, 15, Саратов, Саратовская область"

while True:

    starting_position = input("Введите адрес начальной точки:")
    ending_position = input("Введите конечный адрес")

    if starting_position == "" and ending_position == "":
        starting_position = def_s_pos
        ending_position = def_e_pos
        
    start_latlng = locator.geocode(starting_position)
    end_latlng = locator.geocode(ending_position)
    
    if start_latlng == None or end_latlng == None:
        print("Введены некорректные данные. Повторите ввод")
    else:
        break



/home/kali/.local/lib/python3.10/site-packages/osmnx/utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


In [249]:
mode = "bike"
place = "Саратов, Россия"
optimizer = "drive"

graph = ox.graph_from_place(place, network_type = mode)

In [250]:
start_node = ox.nearest_nodes(graph, X=start_latlng.longitude, Y=start_latlng.latitude)
end_node = ox.nearest_nodes(graph, X=end_latlng.longitude, Y=end_latlng.latitude)

In [251]:
path[str(start_node)] = f"{start_node}->"

In [252]:
for node in graph.nodes():
    distances[str(node)] = math.inf
    edges[str(node)] = []

for u, v, data in graph.edges(data=True):
    edges[str(u)].append((str(v), data["length"]))

    if  data["reversed"]:
        edges[str(v)].append((str(u), data["length"]))


In [253]:
Dijkstra(str(start_node))

In [254]:
l_per_meter = float(input("Расход топлива на 100км: ")) / 100_000
oil_price = float(input("Цена на бензин: "))

In [255]:
short_path = [int(i) for i in path[str(end_node)].split("->") if i != "" ]
print(f"Расстояние на дорогу: {distances[str(end_node)]} м \nЗатраченные деньги на дорогу: {float('{:.2f}'.format(distances[str(end_node)] * l_per_meter * oil_price))} рублей")
shortest_route_map = ox.plot_route_folium(graph, short_path, tiles='openstreetmap')

print()
start_coord = (start_latlng[1][0], start_latlng[1][1])
end_coord = (end_latlng[1][0],end_latlng[1][1])
print(end_latlng)

start_marker = folium.Marker(
            location = start_coord,
            popup = starting_position,
            icon = folium.Icon(color='green'))

end_marker = folium.Marker(
            location = end_coord,
            popup = ending_position,
            icon = folium.Icon(color='red'))

start_marker.add_to(shortest_route_map)
end_marker.add_to(shortest_route_map)

shortest_route_map

Расстояние на дорогу: 2908.489 м 
Затраченные деньги на дорогу: 25.47 рублей

15, Волжская улица, Липки, Волжский район, городской округ Саратов, Саратовская область, Приволжский федеральный округ, 410000, Россия
